# 1. Extract results

In [ ]:
import sys

sys.path.append("../../")
from _analysis._rule_app_analysis import automatic_results, save_results_to_json

In [ ]:
base_path = "../../Data/Temp/Benchmark/"
test_types = ["Valid", "Test"]
temp_types = ["Raw", "Complete", "Expand"]
predict_types = ["fw", "bw"]
radius = [0, 1, 2, 3]
results = automatic_results(test_types, temp_types, predict_types, radius, base_path)

## 2. Barplot

In [ ]:
import sys

sys.path.append("../../")
from _analysis._rule_app_analysis import load_results_from_json

results = load_results_from_json("../../Data/Temp/Benchmark/raw_results.json")

In [ ]:
import pandas as pd

valid = results["Valid"]
valid_fw = valid["fw"]
valid_bw = valid["bw"]
fw = pd.DataFrame(valid_fw).T
bw = pd.DataFrame(valid_bw).T
fw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)
bw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)

In [ ]:
from _analysis._rule_app_analysis import plot_percentage

In [ ]:
import matplotlib.pyplot as plt

# Creating the figure and subplots
plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{amsmath}")  # Ensure amsmath is loaded
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 18))
fontsettings = {
    "title_size": 32,
    "label_size": 28,
    "ticks_size": 28,
    "annotation_size": 18,
}

# Plotting data
plot_percentage(fw, ax1, "C", title=r"A", fontsettings=fontsettings)
plot_percentage(bw, ax2, "C", title=r"B", fontsettings=fontsettings)

plot_percentage(fw, ax3, "FPR", title=r"C", fontsettings=fontsettings)
plot_percentage(bw, ax4, "FPR", title=r"D", fontsettings=fontsettings)


fig.legend(
    [r"$R_0$", r"$R_1$", r"$R_2$", r"$R_3$"],  # Correct LaTeX formatted strings
    loc="lower center",
    ncol=4,
    fontsize=24,
    bbox_to_anchor=(0.5, 0.001),
)

# Adjusting layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.15, bottom=0.08)
fig.savefig(
    "./fig/template_false_rate_compare_valid.pdf",
    dpi=600,
    bbox_inches="tight",
    pad_inches=0,
)

## 3. ROC curve

In [ ]:
import sys
import pandas as pd
import numpy as np

sys.path.append("../../")
from _analysis._rule_app_analysis import load_results_from_json

results = load_results_from_json("../../Data/Temp/Benchmark/raw_results.json")

valid = results["Test"]

valid_fw = valid["fw"]
valid_bw = valid["bw"]
fw = pd.DataFrame(valid_fw).T
bw = pd.DataFrame(valid_bw).T
fw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)
bw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)

fw[["Type", "Radii"]] = fw.index.to_series().str.split("_", expand=True)
bw[["Type", "Radii"]] = bw.index.to_series().str.split("_", expand=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from _analysis._rule_app_analysis import plot_roc_curves

plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{amsmath}")  # Ensure amsmath is loaded
fontsettings = {
    "title_size": 24,
    "label_size": 20,
    "ticks_size": 20,
    "annotation_size": 16,
}
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
plot_roc_curves(
    fw,
    axs[0],
    selected_types=["Complete", "Expand"],
    fontsettings=fontsettings,
    title="A",
)
legend_handles = plot_roc_curves(
    bw,
    axs[1],
    selected_types=["Complete", "Expand"],
    fontsettings=fontsettings,
    title="B",
)

fig.legend(
    handles=legend_handles,
    loc="lower center",
    fancybox=True,
    title_fontsize=fontsettings["label_size"],
    fontsize=fontsettings["annotation_size"],
    ncol=3,
    bbox_to_anchor=(0.5, 0.001),
)

fig.tight_layout()
fig.subplots_adjust(hspace=0.15, wspace=0.2, bottom=0.2)
fig.savefig("./fig/ROC_test.pdf", dpi=600, bbox_inches="tight", pad_inches=0)

## G-mean

In [ ]:
import sys
import pandas as pd
import numpy as np

sys.path.append("../../")
from _analysis._rule_app_analysis import load_results_from_json

results = load_results_from_json("../../Data/Temp/Benchmark/raw_results.json")

valid = results["Valid"]

valid_fw = valid["fw"]
valid_bw = valid["bw"]
fw = pd.DataFrame(valid_fw).T
bw = pd.DataFrame(valid_bw).T
fw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)
bw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)

fw[["Type", "Radii"]] = fw.index.to_series().str.split("_", expand=True)
bw[["Type", "Radii"]] = bw.index.to_series().str.split("_", expand=True)

In [ ]:
def gmean(tpr, fpr):
    tnr = 1 - fpr  # True Negative Rate
    g_mean = np.sqrt(tpr * tnr)
    return g_mean


# Calculate G-mean for each row and add it as a new column
fw["G-mean-forward"] = fw.apply(
    lambda row: gmean(row["C"] / 100, row["FPR"] / 100), axis=1
)
bw["G-mean-backward"] = bw.apply(
    lambda row: gmean(row["C"] / 100, row["FPR"] / 100), axis=1
)

In [ ]:
valid_result = pd.concat(
    [fw["G-mean-forward"], bw[["G-mean-backward", "Type", "Radii"]]], axis=1
)

In [ ]:
test_result = pd.concat(
    [fw["G-mean-forward"], bw[["G-mean-backward", "Type", "Radii"]]], axis=1
)

In [ ]:
def gmean(tpr, fpr):
    tnr = 1 - fpr  # True Negative Rate
    g_mean = np.sqrt(tpr * tnr)
    return g_mean


# Calculate G-mean for each row and add it as a new column
fw["G-mean"] = fw.apply(lambda row: gmean(row["C"] / 100, row["FPR"] / 100), axis=1)
bw["G-mean"] = bw.apply(lambda row: gmean(row["C"] / 100, row["FPR"] / 100), axis=1)

In [ ]:
fw

In [ ]:
bw

In [ ]:
g_mean_results = {}

for type_, group in fw.groupby("Type"):
    tpr = (
        group["C"] / 100
    )  # True Positive Rate (C is already in percentage, so divide by 100)
    fpr = group["FPR"] / 100  # False Positive Rate
    tnr = 1 - fpr  # True Negative Rate
    g_mean = np.sqrt(tpr * tnr).mean()  # Geometric Mean
    g_mean_results[type_] = g_mean

# Output the G-mean results for each Type
g_mean_results

## 4. Time processing

In [ ]:
from datetime import datetime


def calculate_time_difference(log_file_path):
    # Define the time format from the logs
    time_format = "%Y-%m-%d %H:%M:%S,%f"

    # Initialize variables to store timestamps
    start_time = None
    end_time = None

    # Open and read the log file
    with open(log_file_path, "r") as file:
        for line in file:
            # Check if line contains the start process timestamp
            if "Start process" in line:
                timestamp_str = line.split(" - ")[0]
                start_time = datetime.strptime(timestamp_str, time_format)

            # Check if line contains the forward prediction validation timestamp
            elif "Forward Prediction Validation" in line:
                timestamp_str = line.split(" - ")[0]
                end_time = datetime.strptime(timestamp_str, time_format)

    # Ensure both timestamps are found
    if start_time is None or end_time is None:
        raise ValueError(
            "Timestamps for 'Start process' or 'Forward Prediction Validation' not found in the log file."
        )

    # Calculate the time difference
    time_difference = end_time - start_time
    total_seconds = time_difference.total_seconds()

    return total_seconds


# Example usage
log_file_path = "../../Data/Temp/Benchmark/Raw/Log/Test"

In [ ]:
radius = [0, 1, 2, 3]
result = []
for i in radius:
    try:
        total_seconds = calculate_time_difference(
            log_file_path=f"{log_file_path}/r{i}.txt"
        )
        result.append(total_seconds)
    except:
        result.append(None)

In [ ]:
result

In [ ]:
valid_times_compare = {
    r"$Q_{\text{raw}}$": [9169.663, 34687.0292, 212203.163, 450822.167],
    r"$Q_{\text{complete}}$": [10067.556, 36444.666, 213271.507, 458275.105],
    r"$Q_{\text{expand}}$": [10882.231, 36850.825, 215493.644, 465514.313],
}

In [ ]:
test_times_compare = {
    r"$Q_{\text{raw}}$": [9967.84, 34812.77, 214118.684, 460135.812],
    r"$Q_{\text{complete}}$": [5553.019, 17230.426, 170896.315, 450532.96],
    r"$Q_{\text{expand}}$": [11065.185, 36902.738, 215238.352, 466254.943],
}

In [ ]:
from _analysis._rule_app_analysis import plot_roc_curves, plot_processing_times

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 1, figsize=(16, 8))
plot_processing_times(valid_times_compare, ax=axs, title="A. Time benchmarking")
# plot_processing_times(test_times_compare, ax=axs[1], title = 'B')
# fig.savefig('../../time_process_rule', bbox_inches='tight', pad_inches=0)

In [ ]:
import sys
import pandas as pd
import numpy as np

sys.path.append("../../")
from _analysis._rule_app_analysis import load_results_from_json

results = load_results_from_json("../../Data/Temp/Benchmark/raw_results.json")

valid = results["Valid"]

valid_fw = valid["fw"]
valid_bw = valid["bw"]
fw = pd.DataFrame(valid_fw).T
bw = pd.DataFrame(valid_bw).T
fw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)
bw.rename(
    columns={
        0: "average_solution",
        # 1: r'\mathcal(C)',
        1: "C",
        2: "FPR",
    },
    inplace=True,
)

fw[["Type", "Radii"]] = fw.index.to_series().str.split("_", expand=True)
bw[["Type", "Radii"]] = bw.index.to_series().str.split("_", expand=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have functions like plot_roc_curves and plot_processing_times already defined

# Set up font settings and LaTeX for plot text
plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{amsmath}")  # Ensure amsmath is loaded
fontsettings = {
    "title_size": 24,
    "label_size": 20,
    "ticks_size": 20,
    "annotation_size": 16,
}

# Create a 2x2 subplot layout
fig, axs = plt.subplots(
    2, 2, figsize=(14, 15)
)  # Adjusted figure size for better layout

# Plot time processing in the first row, spanning both columns
axs[0, 0].remove()  # Remove the original first subplot in the first row
axs[0, 1].remove()  # Remove the second subplot in the first row
ax_time = fig.add_subplot(2, 2, (1, 2))  # Add a new subplot that spans the first row
plot_processing_times(valid_times_compare, ax=ax_time, title=r"A. Time Benchmarking")

# Plot ROC curves in the second row
legend_handles_fw = plot_roc_curves(
    fw,
    axs[1, 0],
    selected_types=["Complete", "Expand"],
    fontsettings=fontsettings,
    title=r"B. ROC Curves Validation",
)
legend_handles_bw = plot_roc_curves(
    bw,
    axs[1, 1],
    selected_types=["Complete", "Expand"],
    fontsettings=fontsettings,
    title=r"C. ROC Curves Test",
)

# Combine legends from the ROC curves
fig.legend(
    handles=legend_handles_fw,
    loc="lower center",
    fancybox=True,
    title_fontsize=fontsettings["label_size"],
    fontsize=fontsettings["annotation_size"],
    ncol=3,
    bbox_to_anchor=(0.5, 0.05),
    prop={"size": 18},
)

# Adjust layout for better visual display
fig.tight_layout()
fig.subplots_adjust(
    hspace=0.15, wspace=0.2, bottom=0.17
)  # Adjust spacing to accommodate the legend
fig.savefig(
    "../../Docs/Analysis/fig/time_process_rule.pdf",
    dpi=600,
    bbox_inches="tight",
    pad_inches=0,
)
plt.show()